In [33]:
# Install Java, Spark, and Findspark
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!apt-get update
!apt-get install openjdk-8-jdk
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjd

In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLEnvironment").getOrCreate()

In [35]:
from pyspark import SparkFiles
#Import matched_npa_requests.csv file
url_5 = "https://charlotte-rezoning-model.s3.us-east-2.amazonaws.com/matched_npa_requests_year.csv"
spark.sparkContext.addFile(url_5)
npa_data_df = spark.read.csv(SparkFiles.get("matched_npa_requests_year.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
npa_data_df.show()

+---+----------+------------------+---+--------------------+--------+----------+--------------------+----------+----------+----+---+-------+-----+-------------------+-------------------+-----+-----+---------+---------+----+--------+-----------------+----------------+----------------+------------------+----------+----------+--------------------+--------------------+--------+--------------------+----------+--------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+------------------+------------------+--------------------+-----------+----+
|_c0|Unnamed: 0|         npa_acres|NPA|        npa_geometry|OBJECTID|  Petition|          Petitioner|  FromZone|    ToZone|Type|SPA|Overlay|INNOV|              Acres|           PetAcres|Water|Sewer|CityLimit|ZoningMap| ODZ|     CCW|      ServiceArea|       LuseStaff|     UrbanDesign|       RezoneStaff|Consistent|ProtestPet|            Received|            Approved|Decision|    

In [36]:
from pyspark.sql.functions import isnan, isnull, when, count, col

In [37]:
# there are null Decision values....they need to be removed
npa_data_df2 = npa_data_df.withColumn("nullDecisionFlag", when(isnull("Decision"), 0).otherwise(1))
npa_data_df2.filter("nullDecisionFlag = 0").show()
print(npa_data_df2.count())

+---+----------+-------------+---+--------------------+--------+-----------+----------+--------+----------+----+---+-------+-----+-----+--------+-----+-----+---------+---------+----+----+-----------+---------+-----------+-----------+----------+----------+--------+--------------------+--------+---------+----------+--------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+--------------------+------------------+--------------------+-----------+----+----------------+
|_c0|Unnamed: 0|    npa_acres|NPA|        npa_geometry|OBJECTID|   Petition|Petitioner|FromZone|    ToZone|Type|SPA|Overlay|INNOV|Acres|PetAcres|Water|Sewer|CityLimit|ZoningMap| ODZ| CCW|ServiceArea|LuseStaff|UrbanDesign|RezoneStaff|Consistent|ProtestPet|Received|            Approved|Decision|Hyperlink|Corrective|Propluse|AdminAppr|AdminApprD|AdminApprP|AdminApprC|SFLOTS|THUNITS|CONDOS| APT|TOTUNITS|OFFICE|RETAIL|HOTELRMS|PARKING|OPENSP|ACRESITE|    

In [38]:
npa_data_filteredbyyear_df = npa_data_df2.filter("year >= '2014'")
drop_list = []
drop_list.append("_c0")
drop_list.append("Unnamed: 0")
npa_data_filteredbyyear_df = npa_data_filteredbyyear_df.drop(*drop_list)

In [39]:
print(npa_data_filteredbyyear_df.count())
npa_data_filteredbyyear_df.show()

1757
+------------------+---+--------------------+--------+--------+--------------------+--------+---------+----+---+-------+-----+-------------------+-------------------+-----+-----+---------+---------+----+--------+-----------+-----------------+-------------+------------------+----------+----------+--------------------+--------------------+--------+--------------------+----------+---------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+------------------+------------------+--------------------+-----------+----+----------------+
|         npa_acres|NPA|        npa_geometry|OBJECTID|Petition|          Petitioner|FromZone|   ToZone|Type|SPA|Overlay|INNOV|              Acres|           PetAcres|Water|Sewer|CityLimit|ZoningMap| ODZ|     CCW|ServiceArea|        LuseStaff|  UrbanDesign|       RezoneStaff|Consistent|ProtestPet|            Received|            Approved|Decision|           Hyperlink|Corrective| Prop

In [40]:
#Import Housing.csv file
url_3 = "https://charlotte-rezoning-model.s3.us-east-2.amazonaws.com/Housing.csv"
spark.sparkContext.addFile(url_3)
character_data_df = spark.read.csv(SparkFiles.get("Housing.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
character_data_df.show()

+---+---+----+------------+-------------+-----------------------------------+---------------------+----------------------------+-------------------+------------+-----------------------------------+-----------------------+-------------------------+------------------+----------------+----------------------+---------------------+---------------+------------------------+------------------------+-------------+---------------+-------------------+-----------+----------------+------------+------------------+--------------+------------------+----------------+
|_c0|NPA|year|housing_size|housing_units|residential_demolition_permit_units|residential_occupancy|new_residential_permit_units|rental_houses_units|foreclosures|residential_renovation_permit_units|residential_demolitions|residential_occupancy_moe|home_ownership_moe|rental_costs_moe|residential_renovation|single_family_housing|new_residential|housing_violations_total|subsidized_housing_units|rental_houses|housing_density|single_family_units

In [ ]:
joined_df = npa_data_filteredbyyear_df.join(character_data_df,on=["NPA","year"], how="left")

In [ ]:
joined_df.show()

+---+----+------------------+--------------------+--------+--------+--------------------+--------+---------+----+---+-------+-----+-------------------+-------------------+-----+-----+---------+---------+----+--------+-----------+-----------------+-------------+------------------+----------+----------+--------------------+--------------------+--------+--------------------+----------+---------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+------------------+------------------+--------------------+-----------+----------------+----+------------+-------------+-----------------------------------+---------------------+----------------------------+-------------------+------------+-----------------------------------+-----------------------+-------------------------+------------------+----------------+----------------------+---------------------+---------------+------------------------+------------------------+------

In [ ]:
def dropDupeDfCols(df):
    newcols = []
    dupcols = []

    for i in range(len(df.columns)):
        if df.columns[i] not in newcols:
            newcols.append(df.columns[i])
        else:
            dupcols.append(i)

    df = df.toDF(*[str(i) for i in range(len(df.columns))])
    for dupcol in dupcols:
        df = df.drop(str(dupcol))

    return df.toDF(*newcols)

In [ ]:
clean_df = dropDupeDfCols(joined_df)
print(clean_df.count())
print(joined_df.count())

1757
1757


In [42]:
column_null_count_df = clean_df.select([count(when(isnull(c), c)).alias(c) for c in clean_df.columns]).show()

+---+----+---------+------------+--------+--------+----------+--------+------+----+---+-------+-----+-----+--------+-----+-----+---------+---------+----+---+-----------+---------+-----------+-----------+----------+----------+--------+--------+--------+---------+----------+--------+---------+----------+----------+----------+------+-------+------+----+--------+------+------+--------+-------+------+--------+-----------+-------------+--------+-----------+----------------+---+------------+-------------+-----------------------------------+---------------------+----------------------------+-------------------+------------+-----------------------------------+-----------------------+-------------------------+------------------+----------------+----------------------+---------------------+---------------+------------------------+------------------------+-------------+---------------+-------------------+-----------+----------------+------------+------------------+--------------+------------------

In [44]:
# dropping columsn with null counts above 1500
clean_columns1_df = clean_df.select("NPA","year","npa_acres","npa_geometry","OBJECTID","Petition","Petitioner","FromZone","ToZone","Type","SPA","Overlay","INNOV","Acres","PetAcres","Water","Sewer","CityLimit","ZoningMap","CCW","ServiceArea","LuseStaff","UrbanDesign","RezoneStaff","ProtestPet","Received","Approved","Decision","Hyperlink","Propluse","SHAPESTArea","SHAPESTLength","centroid","NPA_Located","nullDecisionFlag","_c0","housing_size","housing_units","residential_demolition_permit_units","residential_occupancy","new_residential_permit_units","rental_houses_units","foreclosures","residential_renovation_permit_units","residential_demolitions","residential_occupancy_moe","home_ownership_moe","residential_renovation","single_family_housing","new_residential","housing_violations_total","subsidized_housing_units","rental_houses","housing_density","single_family_units","housing_age","foreclosed_units","rental_costs","subsidized_housing","home_ownership","housing_violations"

                                    )

clean_columns1_df.filter("Decision <> 'App'").show(200)

+---+----+------------------+--------------------+--------+--------+--------------------+----------+------------+----+---+-------------+-----+------------------+------------------+-----+-----+---------+----------+--------------------+-----------+-----------------+-----------------+------------------+----------+--------------------+--------------------+--------+--------------------+--------+------------------+------------------+--------------------+-----------+----------------+----+------------+-------------+-----------------------------------+---------------------+----------------------------+-------------------+------------+-----------------------------------+-----------------------+-------------------------+------------------+----------------------+---------------------+---------------+------------------------+------------------------+-------------+---------------+-------------------+-----------+----------------+------------+------------------+--------------+------------------+
|NPA|

In [ ]:
#from google.colab import files
#clean_columns1_df.write.csv('engagement_merged.csv', mode='overwrite') 
#files.download('engagement_merged.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>